In [1]:
%matplotlib inline

from fastai import *
from fastai.vision import *

from IPython.display import display
PATH = os.path.abspath('../..')

seed = 12345

In [2]:
df = pd.read_csv(f'{PATH}/catalogs/master.csv')

In [3]:
df.sample(10)

,objID,specObjID,ra,dec,deVAB_r,expAB_r,lnLExp_r,lnLdeV_r,z,zErr,...,d4000_err,bptclass,oh_p2p5,oh_p16,oh_p50,oh_p84,oh_p97p5,lgm_tot_p50,sfr_tot_p50,ab
103373,1237667323799929044,2526620059938351104,200.596185,27.182611,0.743014,0.735880,-316.77390,-2111.4060,0.024467,0.000012,...,0.020154,1,8.645249,8.687798,8.768049,8.811856,8.856243,9.216711,-0.435591,0.735880
65505,1237662223533998125,2239589800342480896,194.743631,38.424452,0.825338,0.826988,-361.11110,-973.7265,0.051520,0.000008,...,0.021368,1,8.762623,8.803021,8.839024,8.869947,8.892822,9.757862,-0.030266,0.826988
128885,1237668496855662671,2823902716663719936,175.973267,18.203593,0.642037,0.678334,-45.83228,-251.4373,0.159859,0.000016,...,0.019565,1,8.902919,8.966475,9.016533,9.089045,9.154526,10.492720,0.823441,0.678334
94228,1237664871900512409,2194525760834791424,150.416586,32.763641,0.468344,0.519291,-89.09326,-1072.2530,0.043969,0.000007,...,0.018013,1,8.690337,8.724379,8.791703,8.816744,8.857265,9.208464,-0.413421,0.519291
24455,1237655108917067885,1077634960027838464,195.193344,60.220864,0.545934,0.585668,-24.71450,-575.6803,0.080327,0.000007,...,0.017753,1,9.060454,9.092499,9.108860,9.117888,9.153283,10.067480,0.180357,0.585668
105665,1237667210515644578,2639144628758013952,149.921174,24.554753,0.188595,0.200000,-30.22561,-286.2563,0.052675,0.000015,...,0.070393,1,8.510814,8.610907,8.718410,8.821393,8.900964,9.736110,-0.357994,0.200000
96688,1237665101679231125,2421836614025111552,226.689173,26.984950,0.682181,0.712387,-827.58150,-665.7916,0.082049,0.000012,...,0.023627,1,9.061811,9.114270,9.172981,9.281918,9.363223,10.780220,0.588921,0.682181
45294,1237657771787485341,1021325120859301888,163.535336,56.321231,0.691217,0.715809,-27.77771,-519.7659,0.075728,0.000008,...,0.014856,1,8.884663,8.940504,8.953751,8.970618,8.978599,9.846697,0.084060,0.715809
139647,1237661463310107002,1767823160394147840,251.349999,22.238182,0.474502,0.492134,-2292.04700,-1506.7940,0.055884,0.000007,...,0.016470,1,9.040863,9.049296,9.090587,9.131371,9.139522,10.797420,0.760238,0.474502
23032,1237655474503811429,917783011376785408,247.155615,43.354261,0.184030,0.208046,-90.08473,-1229.3200,0.046003,0.000008,...,0.019214,1,8.614593,8.680563,8.709820,8.735245,8.801517,9.767421,-0.151222,0.208046


In [2]:
def evaluate_preds(Z_pred, Z_true, comment=''):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111)
    *_, im = ax.hist2d(Z_true.flatten().numpy(), Z_pred.flatten().numpy(), bins=100, range=[[8.2, 9.3], [8.2, 9.3]], cmap='cubehelix_r', alpha=0.7)
    ax.plot([8.2, 9.3], [8.2, 9.3], c='k', ls='--', lw=3)
    ax.set_aspect(aspect='equal')

    ax.text(8.25, 9.15, f'{comment}\n' f'RMSE={root_mean_squared_error(Z_pred, Z_true):.4f}', fontsize=16)

    ax.set_xlabel('Z_{\rm true}', fontsize=12)
    ax.set_ylabel('Z_{\rm pred}', fontsize=12)
    cb = fig.colorbar(im);
    cb.set_label('Number', fontsize=12)


# Load data

In [3]:
train_label_csv = f'{PATH}/catalogs/train.csv'
df = pd.read_csv(train_label_csv)

In [4]:
bs = 128
sz = 128

tfms = get_transforms(do_flip=True, flip_vert=True, max_rotate=15, max_zoom=1.15, p_lighting=0)

data = (ImageList.from_df(path=PATH, 
                          df=df, 
                          folder='train',
                          cols='objID',
                          suffix='.jpg')
        .split_by_rand_pct(0.2, seed=seed)
        .label_from_df(cols='oh_p50', label_cls=FloatList)
        .transform(tfms, size=sz)
        .databunch(bs=bs)
       )

# RESULTS

```
RESNET-34

Experiment    total epochs   pretrain   lowest loss
----------    ------------   --------   -----------
     A             15           yes       0.09146
     
     B             15           yes       0.08704
    B-2            15           no        0.08967
    B-3            40           yes       0.08189
    B-4            40           no        0.08493
    B-5           100           yes       0.08089
    
     C             15           yes       0.09002
     
     D             15           yes       0.08951
     
     E             15           yes       0.08594
    E-2            15           no        0.09000
    E-3            30           yes       0.08488
    
     F             15           yes       0.09019

     G             15           yes
```

# Experiment A: 10x1-epoch cos anneal + 1x5-epoch 1cycle
Note that we are using `pct_start=0` for the first 10 loops while annealing the learning rate each time.

In [5]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=True, ps=[0.25, 0.5], loss_func=root_mean_squared_error)

In [6]:
learn.unfreeze()
for n in range(10):
    learn.fit_one_cycle(1, slice((1-(n**2/100))*1e-3, (1-n/10)*3e-2), pct_start=0)

learn.fit_one_cycle(5, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,time
0,0.386296,0.122144,04:25


epoch,train_loss,valid_loss,time
0,0.329857,0.133680,01:11


epoch,train_loss,valid_loss,time
0,0.305009,0.761204,01:11


epoch,train_loss,valid_loss,time
0,0.271997,0.106095,01:10


epoch,train_loss,valid_loss,time
0,0.250951,0.100799,01:11


...

epoch,train_loss,valid_loss,time
0,0.220112,0.096562,01:11


epoch,train_loss,valid_loss,time
0,0.209793,0.103369,01:11


epoch,train_loss,valid_loss,time
0,0.200335,0.095091,01:11


epoch,train_loss,valid_loss,time
0,0.195695,0.093166,01:11


epoch,train_loss,valid_loss,time
0,0.206211,0.112045,01:11
1,0.192797,0.099005,01:10
2,0.175275,0.097394,01:10
3,0.161896,0.102699,01:10
4,0.155001,0.091458,01:11


# Experiment B: 1x15-epoch 1cycle

In [21]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=True, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()
learn.fit_one_cycle(15, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,time
0,0.895777,0.447834,01:11
1,0.535843,0.129687,01:11
2,0.403542,0.259177,01:11
3,0.377406,0.143210,01:11
4,0.334399,0.290690,01:11
5,0.319577,0.148168,01:11
6,0.277378,0.189192,01:11
7,0.253362,0.113329,01:11
8,0.231079,0.098599,01:11
9,0.216428,0.111048,01:11


# Experiment C: 15x1-epoch 1cycle

In [24]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=True, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()
for n in range(15):
    learn.fit_one_cycle(1, slice((1-(n/15)**2)*1e-4, (1-(n/15)**2)*1e-2))

epoch,train_loss,valid_loss,time
0,0.487408,0.115353,01:10


epoch,train_loss,valid_loss,time
0,0.379072,0.105419,01:10


epoch,train_loss,valid_loss,time
0,0.355806,0.103983,01:10


epoch,train_loss,valid_loss,time
0,0.326684,0.098353,01:10


epoch,train_loss,valid_loss,time
0,0.311732,0.095990,01:10


...

epoch,train_loss,valid_loss,time
0,0.231830,0.090300,01:11


epoch,train_loss,valid_loss,time
0,0.224063,0.089927,01:10


epoch,train_loss,valid_loss,time
0,0.218553,0.089685,01:10


epoch,train_loss,valid_loss,time
0,0.213683,0.089045,01:10


epoch,train_loss,valid_loss,time
0,0.208622,0.090021,01:10


# Experiment D: 15x1-epoch cos anneal

In [25]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=True, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()
for n in range(15):
    learn.fit_one_cycle(1, slice((1-(n/15))*1e-4, (1-n/15)*1e-2), pct_start=0)


epoch,train_loss,valid_loss,time
0,0.470845,0.112324,01:10


epoch,train_loss,valid_loss,time
0,0.374111,0.101818,01:10


epoch,train_loss,valid_loss,time
0,0.343196,0.098715,01:10


epoch,train_loss,valid_loss,time
0,0.327406,0.094742,01:11


epoch,train_loss,valid_loss,time
0,0.311407,0.093997,01:11


...

epoch,train_loss,valid_loss,time
0,0.252760,0.090658,01:10


epoch,train_loss,valid_loss,time
0,0.247964,0.090054,01:10


epoch,train_loss,valid_loss,time
0,0.242390,0.088900,01:10


epoch,train_loss,valid_loss,time
0,0.238673,0.089509,01:10


epoch,train_loss,valid_loss,time
0,0.233537,0.115091,01:11


# Experiment E: 15x1-epoch step-down anneal

In [26]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=True, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()
for n in range(15):
    learn.fit(1, slice((1-(n/15))*1e-4, (1-n/15)*1e-2))


epoch,train_loss,valid_loss,time
0,0.457025,0.220166,01:11


epoch,train_loss,valid_loss,time
0,0.378177,0.209406,01:22


epoch,train_loss,valid_loss,time
0,0.330048,0.128339,01:23


epoch,train_loss,valid_loss,time
0,0.300445,0.213982,01:11


epoch,train_loss,valid_loss,time
0,0.284213,0.365322,01:13


...

epoch,train_loss,valid_loss,time
0,0.184064,0.087990,02:28


epoch,train_loss,valid_loss,time
0,0.178923,0.090890,02:22


epoch,train_loss,valid_loss,time
0,0.168754,0.107033,02:25


epoch,train_loss,valid_loss,time
0,0.166456,0.086221,01:48


epoch,train_loss,valid_loss,time
0,0.159215,0.085944,01:57


# Experiment F: 15x1-epoch step-down anneal with higher rates

In [27]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=True, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()
for n in range(15):
    learn.fit(1, slice((1-(n/15))*3e-4, (1-n/15)*3e-2))


epoch,train_loss,valid_loss,time
0,0.500442,0.451812,02:21


epoch,train_loss,valid_loss,time
0,0.369066,0.260719,02:22


epoch,train_loss,valid_loss,time
0,0.308257,0.378961,02:22


epoch,train_loss,valid_loss,time
0,0.261049,0.156423,02:21


epoch,train_loss,valid_loss,time
0,0.216266,0.969047,02:21


...

epoch,train_loss,valid_loss,time
0,0.141784,0.104190,02:23


epoch,train_loss,valid_loss,time
0,0.135177,0.090703,02:22


epoch,train_loss,valid_loss,time
0,0.129515,0.092297,02:21


epoch,train_loss,valid_loss,time
0,0.123481,0.092996,02:20


epoch,train_loss,valid_loss,time
0,0.116439,0.090192,02:24


# Experiment B-2: 1x15-epoch 1cycle (no pretrain)

In [6]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=False, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()
learn.fit_one_cycle(15, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,time
0,0.966672,0.242243,03:55
1,0.519778,0.200161,01:11
2,0.402694,0.207183,01:11
3,0.363541,0.125407,01:11
4,0.340362,0.242408,01:11
5,0.315441,0.192592,01:11
6,0.281019,0.141952,01:11
7,0.258993,0.208171,01:11
8,0.238654,0.100489,01:11
9,0.217802,0.102762,01:11


# Experiment B-3: 1x40 1cycle (with pretrain)

In [8]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=True, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()
learn.fit_one_cycle(40, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,time
0,1.123706,0.402679,01:11
1,0.688380,0.180077,01:11
2,0.515301,0.128502,01:11
3,0.448472,0.284959,01:11
4,0.384373,0.196770,01:11
5,0.362661,0.146871,01:11
6,0.334832,0.102702,01:11
7,0.320537,0.173960,01:11
8,0.297543,0.130390,01:11
9,0.270987,0.144441,01:11


# Experiment B-4: 1x40 1cycle (no pretrain)

In [9]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=False, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()
learn.fit_one_cycle(40, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,time
0,1.182091,0.237660,01:12
1,0.707433,0.195599,01:11
2,0.546309,0.357620,01:11
3,0.444520,0.132701,01:11
4,0.394261,0.124060,01:11
5,0.357019,0.128260,01:11
6,0.334078,0.142870,01:11
7,0.326543,0.117135,01:11
8,0.297568,0.230253,01:11
9,0.273850,0.123156,01:11


# Experiment B-5: 1x100 1cycle (pretrain)

In [5]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=True, ps=[0.25, 0.5], loss_func=root_mean_squared_error)

n_epoch = 100

learn.unfreeze()
learn.fit_one_cycle(n_epoch, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,time
0,1.142310,0.388418,04:20
1,0.659236,0.213387,01:11
2,0.555667,0.169565,01:11
3,0.483397,0.115337,01:11
4,0.450553,0.109885,01:11
5,0.416657,0.109158,01:11
6,0.394183,0.109840,01:11
7,0.377036,0.131921,01:11
8,0.362325,0.142010,01:11
9,0.337034,0.117399,01:11


In [6]:
learn.save('resnet34-experiment_B5')

# Experiment G: 1x15 cos anneal (higher initial rate)

In [6]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=True, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()

n_epoch = 15
learn.fit_one_cycle(n_epoch, slice(1e-3, 1e-1), pct_start=0)

epoch,train_loss,valid_loss,time
0,1.916937,2.443507,02:31


KeyboardInterrupt: 

# Experiment E-2: 15x1 step-down anneal (no pretrain)

In [7]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=False, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()

n_epochs = 15
for n in range(n_epochs):
    learn.fit(1, slice((1-(n/n_epochs)**2)*1e-4, (1-(n/n_epochs)**2)*1e-2))


epoch,train_loss,valid_loss,time
0,0.427407,0.154175,01:11


epoch,train_loss,valid_loss,time
0,0.369980,0.161240,01:11


epoch,train_loss,valid_loss,time
0,0.333630,0.175207,01:11


epoch,train_loss,valid_loss,time
0,0.307659,0.129259,01:11


epoch,train_loss,valid_loss,time
0,0.278915,0.144768,01:11


...

epoch,train_loss,valid_loss,time
0,0.190107,0.114193,01:11


epoch,train_loss,valid_loss,time
0,0.183839,0.098890,01:11


epoch,train_loss,valid_loss,time
0,0.173237,0.095118,01:11


epoch,train_loss,valid_loss,time
0,0.167197,0.090003,01:11


epoch,train_loss,valid_loss,time
0,0.164201,0.098844,01:11


# Experiment E-3: 30x1 step-down anneal (pretrained)

In [7]:
learn = cnn_learner(data, base_arch=models.resnet34, pretrained=True, ps=[0.25, 0.5], loss_func=root_mean_squared_error)
learn.unfreeze()

n_epochs = 30
for n in range(n_epochs):
    learn.fit(1, slice((1-(n/n_epochs)**2)*1e-4, (1-(n/n_epochs)**2)*1e-2))


epoch,train_loss,valid_loss,time
0,0.419458,0.145207,02:44


epoch,train_loss,valid_loss,time
0,0.357330,0.179197,02:41


epoch,train_loss,valid_loss,time
0,0.329022,0.127584,01:52


epoch,train_loss,valid_loss,time
0,0.292913,0.262084,02:10


epoch,train_loss,valid_loss,time
0,0.263237,0.141279,02:12


...

epoch,train_loss,valid_loss,time
0,0.105156,0.093919,02:28


epoch,train_loss,valid_loss,time
0,0.104253,0.085419,02:21


epoch,train_loss,valid_loss,time
0,0.102944,0.100398,02:31


epoch,train_loss,valid_loss,time
0,0.100750,0.089119,02:31


epoch,train_loss,valid_loss,time
0,0.100640,0.084881,02:35
